In [1]:
import pandas as pd
import pymer4.models
import numpy as np
import scipy.stats
import os

# Output and formatting settings

In [2]:
column_names = {"Estimate": "$\\beta$", "Z-stat": "$z$", "P-val": "$p$", "T-stat": "$t$", "F-stat": "$F$", "2.5_ci": "CI 2.5\%", "97.5_ci": "CI 97.5\%", "NumDF": "df"}
var_names = {"tta_z": "$\\textrm{TTA}$",
             "d_z": "distance",
             "time_budget_z": "time budget",
             "decision1": "decision",
             "dwell_mirror_z": "\% dwell time mirror",
             "RT_z": "RT",
             "tta_z:time_budget_z": "$\\textrm{TTA}$:time budget",
             "d_z:dwell_mirror_z": "distance:\% dwell time mirror",
             "tta_z:dwell_mirror_z": "$\\textrm{TTA}$:\% dwell time mirror",
             "time_budget_z:dwell_mirror_z": "time budget:\% dwell time mirror",
             "tta_z:time_budget_z:dwell_mirror_z": "$\\textrm{TTA}$:time budget:\% dwell time mirror",
             "decision1:d_z": "decision:distance",
             "decision1:tta_z": "decision:$\\textrm{TTA}$",
             "decision1:time_budget_z": "decision:time budget",
             "decision1:tta_z:time_budget_z": "decision:$\\textrm{TTA}$:time budget"}

def p_formatted(p):
    if p>0.01:
        return "{:.3f}".format(p)
    elif p>0.001:
        return "{:.3f}".format(p)
    else:
        return "$<0.001$"

# Read and pre-process the data

In [3]:
output_path = "output"

processed_data_path = "../../surfdrive/data/merging_eye_tracking/processed"
data = pd.read_csv(os.path.join(processed_data_path, "processed_eye_data.csv"))
metrics = pd.read_csv(os.path.join(processed_data_path, "metrics.csv"))

def get_z_score(x):
    return (x-x.mean())/x.std()

for col in ["RT", "d", "tta", "time_budget", "dwell_mirror"]:
    metrics.loc[:, col+"_z"] = get_z_score(metrics[col]) 

# Decision outcome as a function of kinematic conditions

In [4]:
model_decision = pymer4.models.Lmer("is_gap_accepted ~ 1 + d_z + tta_z + time_budget_z + (1 + tta_z + time_budget_z + d_z | participant)", data=metrics, family="binomial")
model_decision_fit = model_decision.fit(summarize=True)
model_decision.coefs

Linear mixed model fit by maximum likelihood  ['lmerMod']
Formula: is_gap_accepted~1+d_z+tta_z+time_budget_z+(1+tta_z+time_budget_z+d_z|participant)

Family: binomial	 Inference: parametric

Number of observations: 8634	 Groups: {'participant': 24.0}

Log-likelihood: -3232.833 	 AIC: 6493.667

Random effects:

                      Name    Var    Std
participant    (Intercept)  1.622  1.273
participant          tta_z  0.268  0.518
participant  time_budget_z  0.202  0.449
participant            d_z  0.251  0.501

                       IV1            IV2   Corr
participant    (Intercept)          tta_z -0.124
participant    (Intercept)  time_budget_z -0.220
participant    (Intercept)            d_z  0.215
participant          tta_z  time_budget_z -0.140
participant          tta_z            d_z -0.548
participant  time_budget_z            d_z  0.462

Fixed effects:


,Estimate,2.5_ci,97.5_ci,SE,OR,OR_2.5_ci,OR_97.5_ci,Prob,Prob_2.5_ci,Prob_97.5_ci,Z-stat,P-val,Sig
(Intercept),1.299496,0.780144,1.818847,0.264980,3.667447,2.181787,6.164747,0.785750,0.685711,0.860428,4.904127,9.384372e-07,***
d_z,0.537354,0.325556,0.749153,0.108062,1.711472,1.384800,2.115207,0.631197,0.580678,0.678994,4.972624,6.605256e-07,***
tta_z,1.841692,1.611944,2.071439,0.117220,6.307198,5.012548,7.936233,0.863149,0.833681,0.888096,15.711401,1.263553e-55,***
time_budget_z,-0.416696,-0.608421,-0.224971,0.097821,0.659221,0.544210,0.798539,0.397308,0.352420,0.443993,-4.259795,2.046146e-05,***


In [5]:
coefs = model_decision.coefs.loc[:, ["Estimate", "SE", "Z-stat", "P-val"]]
coefs["P-val"] = coefs["P-val"].apply(p_formatted)
styler = coefs.rename(columns=column_names, index=var_names).style.format(precision=2)

with open(os.path.join(output_path, "tab_decision.tex"), 'w') as writer:
     writer.write(styler.to_latex(
         column_format="rrrrr", position="!t", position_float="centering",
         hrules=True, label="tab:decision", caption="Standardized coefficients of the mixed-effects logistic regression describing the final decision. All effects were modelled as random slopes per participant: \\texttt{decision $\sim$ 1 + distance + TTA + time budget + (1 + distance + TTA + time budget) | participant}."
     )
)

# Response time

In [6]:
model_RT = pymer4.models.Lmer("RT_z ~ 1 + decision*(d_z + tta_z*time_budget_z) + (decision | participant) ", data=metrics, family="gaussian")
model_RT.fit(summarize=True, factors={"decision": ["Accept", "Reject"]})
model_RT.coefs

Linear mixed model fit by REML [’lmerMod’]
Formula: RT_z~1+decision*(d_z+tta_z*time_budget_z)+(decision|participant)

Family: gaussian	 Inference: parametric

Number of observations: 8634	 Groups: {'participant': 24.0}

Log-likelihood: -9164.353 	 AIC: 18356.706

Random effects:

                       Name    Var    Std
participant     (Intercept)  0.343  0.586
participant  decisionReject  0.201  0.449
Residual                     0.475  0.689

                     IV1             IV2   Corr
participant  (Intercept)  decisionReject -0.431

Fixed effects:


,Estimate,2.5_ci,97.5_ci,SE,DF,T-stat,P-val,Sig
(Intercept),-0.262322,-0.497570,-0.027074,0.120027,23.082381,-2.185533,3.925263e-02,*
decision1,1.076628,0.891219,1.262037,0.094598,23.988532,11.381083,3.732780e-11,***
d_z,-0.036674,-0.054909,-0.018440,0.009303,8579.888190,-3.942068,8.142388e-05,***
tta_z,0.038983,0.018668,0.059298,0.010365,8589.916562,3.761069,1.703127e-04,***
time_budget_z,0.057298,0.037564,0.077032,0.010069,8579.983000,5.690774,1.306221e-08,***
tta_z:time_budget_z,0.044440,0.024741,0.064139,0.010051,8579.076836,4.421580,9.918840e-06,***
decision1:d_z,0.091484,0.059867,0.123102,0.016132,8591.988758,5.671088,1.464876e-08,***
decision1:tta_z,0.284746,0.241664,0.327829,0.021981,8599.404594,12.954012,5.051654e-38,***
decision1:time_budget_z,0.131781,0.089988,0.173575,0.021323,8583.265825,6.180098,6.698096e-10,***
decision1:tta_z:time_budget_z,0.042008,0.000436,0.083580,0.021211,8581.322753,1.980505,4.767869e-02,*


In [7]:
coefs = model_RT.coefs.loc[:, ["Estimate", "SE", "DF", "T-stat", "P-val"]]
coefs["P-val"] = coefs["P-val"].apply(p_formatted)
styler = coefs.rename(columns=column_names, index=var_names).style.format(precision=2)

with open(os.path.join(output_path, "tab_RT.tex"), 'w') as writer:
     writer.write(styler.to_latex(
         column_format="rrrrrr", position="!t", position_float="centering",
         hrules=True, label="tab:RT", caption="Standardized coefficients of the mixed-effects linear regression describing response times. Random slope of decision was included per participant: \\texttt{RT $\sim$ 1 + decision*(TTA*time budget + distance) + (1 + decision) | participant}. ``Accept'' was set as a reference level for the decision outcome factor. Degrees of freedom were estimated using the  Satterthwaite approximation."
     )
)

## ANOVA

In [8]:
RT_anova = model_RT.anova()
RT_decision_marginal_estimates, RT_decision_comparisons = model_RT.post_hoc(marginal_vars=["decision"])
RT_anova

SS Type III Analysis of Variance Table with Satterthwaite approximated degrees of freedom:
(NOTE: Using original model contrasts, orthogonality not guaranteed)


,SS,MS,NumDF,DenomDF,F-stat,P-val,Sig
decision,61.469085,61.469085,1,23.988532,129.529044,3.732780e-11,***
d_z,0.599919,0.599919,1,8594.728327,1.264164,2.608958e-01,
tta_z,129.168121,129.168121,1,8597.057317,272.185979,3.173882e-60,***
time_budget_z,63.368563,63.368563,1,8586.658892,133.531667,1.165224e-30,***
tta_z:time_budget_z,18.071613,18.071613,1,8582.008689,38.080911,7.094581e-10,***
decision:d_z,15.262386,15.262386,1,8591.988758,32.161243,1.464876e-08,***
decision:tta_z,79.633941,79.633941,1,8599.404594,167.806438,5.051654e-38,***
decision:time_budget_z,18.125095,18.125095,1,8583.265825,38.193609,6.698096e-10,***
decision:tta_z:time_budget_z,1.861408,1.861408,1,8581.322753,3.922400,4.767869e-02,*


In [9]:
coefs = RT_anova.loc[:, ["SS", "MS", "F-stat", "P-val"]]
coefs["P-val"] = coefs["P-val"].apply(p_formatted)
styler = coefs.rename(columns=column_names, index=var_names).style.format(precision=2)

with open(os.path.join(output_path, "tab_RT_ANOVA.tex"), 'w') as writer:
     writer.write(styler.to_latex(
         column_format="rrrrrr", position="!t", position_float="centering",
         hrules=True, label="tab:RT_ANOVA", caption="ANOVA table based on the mixed-effects linear regression describing response time. Random slope of decision was included per participant: \\texttt{RT $\sim$ 1 + decision*(TTA*time budget + distance) + (1 + decision) | participant}."
     )
)

## Difference between accept and reject RTs

In [10]:
RT_decision_marginal_estimates, RT_decision_comparisons = model_RT.post_hoc(marginal_vars=["decision"])
RT_decision_comparisons

,Contrast,Estimate,2.5_ci,97.5_ci,SE,DF,T-stat,P-val,Sig
1,Accept - Reject,-1.077,-1.272,-0.881,0.095,23.988,-11.382,0.0,***


In [11]:
RT_decision_comparisons.Estimate*metrics.RT.std()

1   -0.687359
Name: Estimate, dtype: float64

## Estimates of condition effects on RT per decision

In [12]:
def get_marginal_estimates(model_RT, marginal_vars):
    marginal_estimates, comparisons = model_RT.post_hoc(marginal_vars=marginal_vars, grouping_vars=["decision"])
    marginal_estimates["T-stat"] = marginal_estimates["Estimate"]/marginal_estimates["SE"]
    marginal_estimates["P-val"] = scipy.stats.t.sf(np.abs(marginal_estimates["T-stat"]), marginal_estimates.DF)
    return marginal_estimates

In [13]:
get_marginal_estimates(model_RT, "tta_z")

,decision,Estimate,2.5_ci,97.5_ci,SE,DF,T-stat,P-val
1,Accept,0.039,0.019,0.059,0.010,8589.916,3.900000,4.846282e-05
2,Reject,0.324,0.286,0.362,0.019,8598.869,17.052632,1.880770e-64


In [14]:
get_marginal_estimates(model_RT, "d_z")

,decision,Estimate,2.5_ci,97.5_ci,SE,DF,T-stat,P-val
1,Accept,-0.037,-0.055,-0.018,0.009,8579.888,-4.111111,0.000020
2,Reject,0.055,0.029,0.081,0.013,8597.518,4.230769,0.000012


In [15]:
get_marginal_estimates(model_RT, "time_budget_z")

,decision,Estimate,2.5_ci,97.5_ci,SE,DF,T-stat,P-val
1,Accept,0.057,0.038,0.077,0.010,8579.983,5.700000,6.188545e-09
2,Reject,0.189,0.152,0.226,0.019,8586.301,9.947368,1.727870e-23


## Estimates of time budget effect per decision and TTA level

In [16]:
marginal_estimates, comparisons = model_RT.post_hoc(marginal_vars=["time_budget_z"], grouping_vars=["decision", "tta_z"])
marginal_estimates["T-stat"] = marginal_estimates["Estimate"]/marginal_estimates["SE"]
marginal_estimates["P-val"] = scipy.stats.t.sf(np.abs(marginal_estimates["T-stat"]), marginal_estimates.DF)

In [17]:
marginal_estimates

,decision,tta_z,Estimate,2.5_ci,97.5_ci,SE,DF,T-stat,P-val
1,Accept,-1.0,0.013,-0.020,0.046,0.017,8580.044,0.764706,2.222339e-01
2,Reject,-1.0,0.103,0.075,0.130,0.014,8590.524,7.357143,1.025880e-13
3,Accept,1.0,0.102,0.080,0.123,0.011,8578.325,9.272727,1.129338e-20
4,Reject,1.0,0.276,0.207,0.344,0.035,8583.289,7.885714,1.755460e-15


# Decision outcome as a function of dwell time

## Not controlling for RT

In [18]:
model_decision_dwell = pymer4.models.Lmer("is_gap_accepted ~ (d_z + tta_z + time_budget_z)*dwell_mirror_z + (tta_z + time_budget_z | participant) ", data=metrics, family="binomial")
model_decision_dwell_fit = model_decision_dwell.fit(summarize=True)
model_decision_dwell.coefs

Linear mixed model fit by maximum likelihood  ['lmerMod']
Formula: is_gap_accepted~(d_z+tta_z+time_budget_z)*dwell_mirror_z+(tta_z+time_budget_z|participant)

Family: binomial	 Inference: parametric

Number of observations: 8634	 Groups: {'participant': 24.0}

Log-likelihood: -3297.142 	 AIC: 6622.285

Random effects:

                      Name    Var    Std
participant    (Intercept)  1.551  1.245
participant          tta_z  0.331  0.576
participant  time_budget_z  0.136  0.369

                     IV1            IV2   Corr
participant  (Intercept)          tta_z -0.017
participant  (Intercept)  time_budget_z -0.212
participant        tta_z  time_budget_z -0.348

Fixed effects:


,Estimate,2.5_ci,97.5_ci,SE,OR,OR_2.5_ci,OR_97.5_ci,Prob,Prob_2.5_ci,Prob_97.5_ci,Z-stat,P-val,Sig
(Intercept),1.244753,0.736987,1.752518,0.259069,3.472076,2.089631,5.769110,0.776390,0.676337,0.852270,4.804721,1.549675e-06,***
d_z,0.532221,0.468510,0.595932,0.032506,1.702709,1.597611,1.814721,0.630001,0.615031,0.644725,16.372888,2.986964e-60,***
tta_z,1.826713,1.576004,2.077422,0.127915,6.213428,4.835594,7.983856,0.861370,0.828638,0.888689,14.280677,2.887836e-46,***
time_budget_z,-0.417957,-0.579957,-0.255957,0.082654,0.658391,0.559923,0.774175,0.397006,0.358943,0.436358,-5.056681,4.266155e-07,***
dwell_mirror_z,0.012679,-0.087313,0.112671,0.051017,1.012760,0.916391,1.119264,0.503170,0.478186,0.528138,0.248531,8.037235e-01,
d_z:dwell_mirror_z,-0.056011,-0.121570,0.009548,0.033449,0.945529,0.885530,1.009593,0.486001,0.469645,0.502387,-1.674523,9.402792e-02,.
tta_z:dwell_mirror_z,0.054649,-0.043354,0.152652,0.050002,1.056170,0.957572,1.164920,0.513659,0.489163,0.538089,1.092930,2.744245e-01,
time_budget_z:dwell_mirror_z,0.251792,0.165288,0.338295,0.044135,1.286328,1.179733,1.402554,0.562617,0.541228,0.583776,5.705026,1.163252e-08,***


## Controlling for RT

In [19]:
model_decision_dwell_RT = pymer4.models.Lmer("is_gap_accepted ~ (d_z + tta_z + time_budget_z)*dwell_mirror_z + RT_z + (d_z + tta_z + time_budget_z | participant) ", data=metrics, family="binomial")
model_decision_dwell_RT_fit = model_decision_dwell_RT.fit(summarize=True)
model_decision_dwell_RT.coefs

Linear mixed model fit by maximum likelihood  ['lmerMod']
Formula: is_gap_accepted~(d_z+tta_z+time_budget_z)*dwell_mirror_z+RT_z+(d_z+tta_z+time_budget_z|participant)

Family: binomial	 Inference: parametric

Number of observations: 8634	 Groups: {'participant': 24.0}

Log-likelihood: -2434.363 	 AIC: 4906.727

Random effects:

                      Name    Var    Std
participant    (Intercept)  2.475  1.573
participant            d_z  0.316  0.562
participant          tta_z  0.542  0.736
participant  time_budget_z  0.106  0.325

                     IV1            IV2   Corr
participant  (Intercept)            d_z -0.308
participant  (Intercept)          tta_z  0.418
participant  (Intercept)  time_budget_z -0.451
participant          d_z          tta_z -0.722
participant          d_z  time_budget_z  0.495
participant        tta_z  time_budget_z -0.170

Fixed effects:


,Estimate,2.5_ci,97.5_ci,SE,OR,OR_2.5_ci,OR_97.5_ci,Prob,Prob_2.5_ci,Prob_97.5_ci,Z-stat,P-val,Sig
(Intercept),1.774379,1.131468,2.417290,0.328022,5.896618,3.100204,11.215423,0.855001,0.756110,0.918136,5.409332,6.326039e-08,***
d_z,0.505404,0.266713,0.744094,0.121783,1.657655,1.305666,2.104534,0.623728,0.566286,0.677890,4.150036,3.324224e-05,***
tta_z,2.240503,1.917488,2.563518,0.164807,9.398058,6.803846,12.981408,0.903828,0.871858,0.928476,13.594738,4.302972e-42,***
time_budget_z,-0.279482,-0.431422,-0.127542,0.077522,0.756175,0.649585,0.880256,0.430581,0.393787,0.468158,-3.605208,3.119032e-04,***
dwell_mirror_z,-0.318720,-0.447291,-0.190149,0.065599,0.727079,0.639358,0.826836,0.420988,0.390005,0.452605,-4.858643,1.181930e-06,***
RT_z,-1.967265,-2.092053,-1.842477,0.063669,0.139839,0.123433,0.158425,0.122683,0.109872,0.136759,-30.898494,1.251423e-209,***
d_z:dwell_mirror_z,-0.153212,-0.259814,-0.046611,0.054390,0.857948,0.771195,0.954459,0.461772,0.435409,0.488349,-2.816940,4.848361e-03,**
tta_z:dwell_mirror_z,0.125021,0.003527,0.246515,0.061988,1.133173,1.003534,1.279559,0.531215,0.500882,0.561319,2.016870,4.370908e-02,*
time_budget_z:dwell_mirror_z,0.243566,0.144792,0.342339,0.050395,1.275790,1.155800,1.408237,0.560592,0.536135,0.584759,4.833092,1.344284e-06,***


In [20]:
coefs = model_decision_dwell_RT.coefs.loc[:, ["Estimate", "SE", "Z-stat", "P-val"]]
coefs["P-val"] = coefs["P-val"].apply(p_formatted)
styler = coefs.rename(columns=column_names, index=var_names).style.format(precision=2)

with open(os.path.join(output_path, "tab_decision_dwell_RT.tex"), 'w') as writer:
     writer.write(styler.to_latex(
         column_format="rrrrr", position="!ht", position_float="centering",
         hrules=True, label="tab:decision_dwell_RT", caption="Standardized coefficients of the mixed-effects logistic regression describing the decision outcome as a function of kinematic variables, response time, and relative dwell time. Random slopes of distance and TTA to the overtaking vehicle and the time budget provided by the merging lane were included per participant: \\texttt{decision $\sim$ 1 + RT + distance + (TTA * time budget * \\% dwell time mirror) + (1 + distance + TTA + time budget) | participant}."
     )
)

## ANOVA

In [21]:
decision_dwell_ANOVA = model_decision_dwell_RT.anova()
decision_dwell_marginal_estimates, decision_dwell_comparisons = model_decision_dwell_RT.post_hoc(grouping_vars=["tta_z"], marginal_vars=["time_budget_z"])
decision_dwell_ANOVA

SS Type III Analysis of Variance Table with Satterthwaite approximated degrees of freedom:
(NOTE: Using original model contrasts, orthogonality not guaranteed)


D:\source\daddm\venv\lib\site-packages\pymer4\models\Lmer.py:260: UserWarning: MODELING FIT WARNING! Check model.warnings!! P-value computation did not occur because lmerTest choked. Possible issue(s): ranefx have too many parameters or too little variance...
  warnings.warn(


,DF,SS,MS,F-stat
d_z,1,189.372300,189.372300,189.372300
tta_z,1,73.176075,73.176075,73.176075
time_budget_z,1,15.350488,15.350488,15.350488
dwell_mirror_z,1,0.002303,0.002303,0.002303
RT_z,1,965.465310,965.465310,965.465310
d_z:dwell_mirror_z,1,10.399702,10.399702,10.399702
tta_z:dwell_mirror_z,1,4.513795,4.513795,4.513795
time_budget_z:dwell_mirror_z,1,24.928846,24.928846,24.928846
